In [ ]:
from dataclasses import dataclass, field
from abc import abstractmethod, ABC
import random

class PriorityQueue:
    def __init__(self):
        self.data = []

    def push(self, item):
        self.data.append(item)
        self.data.sort(key=lambda x: x[0])

    def pop(self):
        if self.data:
            return self.data.pop(0)
        else:
            print("Ta vacio papi, paila")

    def __len__(self):
        return len(self.data)

@dataclass
class Player:
    Name : str = "🤸"
    Position: list[int] = field(default_factory = list)
    BestRoute: list[list] = field(default_factory=list)
    IsRetard: bool = False
    IsDumb: bool = False
    History: list[str] = field(default_factory=list)

@dataclass
class SpaceItems(ABC):
    Name: str
    Position: list[int] = field(default_factory=list)


    @abstractmethod
    def Idk():
        pass

@dataclass
class FinishLine(SpaceItems):
    Name: str = "🏁"

    def Idk():
        pass

@dataclass
class Walls(SpaceItems):
    Name: str =  "🧱"

    def Idk():
        pass

@dataclass
class Traps(SpaceItems):
    Name: str = "🪤"
    def Idk():
        pass
        
@dataclass 
class Retarder(SpaceItems):
    Name: str = "🍌"

    def Idk():
        pass


@dataclass
class Space:
    GridSize: int
    FinishLine: FinishLine
    NumberOfPlayers: int
    NumberOfWalls: int
    Trash: int
    Grid: list[list] = field(default_factory=list)
    PlayerList: list[Player] = field(default_factory=list)
    WallsList: list[Walls] = field(default_factory=list)
    TrashList: list[Retarder | Traps] = field(default_factory=list)
    Iteration: int = 0 

    def StartSim(self):
        self.Grid = [[None for _ in range(self.GridSize)] for _ in range(self.GridSize)]
        self.Grid[self.FinishLine.Position[0]][self.FinishLine.Position[1]] = self.FinishLine
        self.CreateTrash()
        self.CreateWalls()
        self.CreatePlayers()

        for wall in self.WallsList:
            self.Grid[wall.Position[0]][wall.Position[1]] = wall
        
        for trash in self.TrashList:
            self.Grid[trash.Position[0]][trash.Position[1]] = trash

        for Player in self.PlayerList:
            self.Grid[Player.Position[0]][Player.Position[1]] = Player



    def UpdateFinishLine(self, NewPos:list[int]):
        self.Grid[self.FinishLine.Position[0]][self.FinishLine.Position[1]] = NewPos


    def UpdateSim(self):
        self.MovePlayers()
        self.AddWall([random.randint(0, self.GridSize - 1), random.randint(0, self.GridSize - 1)])
        self.Grid = [[None for _ in range(self.GridSize)] for _ in range(self.GridSize)]
        self.Grid[self.FinishLine.Position[0]][self.FinishLine.Position[1]] = self.FinishLine
        for wall in self.WallsList:
            self.Grid[wall.Position[0]][wall.Position[1]] = wall
        
        for trash in self.TrashList:
            self.Grid[trash.Position[0]][trash.Position[1]] = trash
        for Player in self.PlayerList:
            self.Grid[Player.Position[0]][Player.Position[1]] = Player


    def ShowGrid(self):
        if self.Iteration == 0:
            print("Primera Iteracion")
            self.StartSim()
        else:
            self.UpdateSim()
        for row in self.Grid:
            print(" ".join(cell.Name if cell else "⬜" for cell in row))
        self.Iteration += 1

    def CreatePlayers(self):
        for _ in range(self.NumberOfPlayers):
            self.PlayerList.append(Player(Position=[random.randint(0, self.GridSize-1), random.randint(0, self.GridSize-1)]))

    def CreateWalls(self):
        for _ in range(self.NumberOfWalls):
                pos = [random.randint(0, self.GridSize-1), random.randint(0, self.GridSize-1)]
                if (pos != self.FinishLine.Position and all(player.Position != pos for player in self.PlayerList)):
                    self.WallsList.append(Walls(Position=pos))

    def CreateTrash(self):
        PosibleItem: list = [Traps, Retarder]
        for _ in range(self.Trash):
                pos = [random.randint(0, self.GridSize-1), random.randint(0, self.GridSize-1)]
                if (pos != self.FinishLine.Position and all(player.Position != pos for player in self.PlayerList)):
                    RandomItem = random.choice(PosibleItem)
                    self.TrashList.append(RandomItem(Position=pos))
    
    def AddWall(self, WallPos:list[int]):
        if (WallPos != self.FinishLine.Position and all(player.Position != WallPos for player in self.PlayerList)):
            wall = Walls(Position=WallPos)
            self.WallsList.append(wall)
        else:
            print("No se puede ubicar el bloqueo, ya existe un elemento alli")




    def MovePlayers(self):
        for player in self.PlayerList:
            directions = [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]
            player.BestRoute = self.CalculateBestRoute(player.Position)

            if (isinstance(player.BestRoute, list) and len(player.BestRoute) > 1 and
            isinstance(player.BestRoute[1], list) and len(player.BestRoute[1]) == 2):
                if not player.IsDumb and not player.IsRetard:
                    OldX, OldY = player.Position
                    NewX, NewY = player.BestRoute[1]
                    cell = self.Grid[NewX][NewY]
                if player.IsDumb:
                    print("Jugador tonto jjijiji")
                    OldX, OldY = player.Position
                    NewX, NewY = player.Position
                    player.History.append(f"me cai con un banano pos{NewX} en X y {NewY} en Y")
                    NewX + random.randint(0,1)
                    NewY = random.randint(0,1)
                    cell = self.Grid[NewX][NewY]
                    player.IsDumb = False
                if player.IsRetard:
                    print("El jugador se encontro una trampa")
                    player.History.append(f"me encontre con una trampa")
                    player.IsRetard = False
                    continue

                if cell is None or isinstance(cell, FinishLine) or isinstance(cell, SpaceItems):
                    self.Grid[OldX][OldY] = None
                    if isinstance(self.Grid[NewX][NewY], Traps):
                        player.IsDumb = True
                    elif isinstance(self.Grid[NewX][NewY], Retarder):
                        player.IsRetard = True
                    player.Position = [NewX, NewY]
                    self.Grid[NewX][NewY] = player
                    player.History.append(f"me he moviod a {player.Position}")

            elif player.BestRoute and player.BestRoute[-1] == self.FinishLine.Position:
                self.PlayerList.remove(player)
                print(f"Un jugador alcanzo la meta!")
                player.History.append(f"He terminado la carrera :)")



    def CalculateBestRoute(self, ActualPos:list[int]):
        directions = [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]
        visited = set()
        pq = PriorityQueue()
        pq.push((0, ActualPos, [ActualPos]))
        finish = self.FinishLine.Position
    
        while len(pq) > 0:
            cost, pos, path = pq.pop()
            pos_tuple = tuple(pos)
            if pos_tuple in visited:
                continue
            visited.add(pos_tuple)
    
            if pos == finish:
                return path  # Camino encontrado
    
            for dx, dy in directions:
                nx, ny = pos[0] + dx, pos[1] + dy
                if 0 <= nx < self.GridSize and 0 <= ny < self.GridSize:
                    cell = self.Grid[nx][ny]
                    # No visitar muros
                    if not isinstance(cell, Walls) and (cell is None or isinstance(cell, FinishLine) or isinstance(cell, SpaceItems)):
                        if (nx, ny) not in visited:
                            pq.push((cost+1, [nx, ny], path + [[nx, ny]]))
        print("ay muchachos")
        return []
    





In [757]:
espacio = Space(15, FinishLine(Position=[7,7]), 2, 50, 20)
# Positions surrounding (7,7)
#surrounding_positions = [
#    [6,6], [6,7], [6,8],
#    [7,6],        [7,8],
#    [8,6], [8,7], [8,8]
#]
#
#for pos in surrounding_positions:
#    espacio.AddWall(WallPos=pos)
#


In [763]:
espacio.ShowGrid()
print(espacio.FinishLine)

Bueno
El jugador se encontro una trampa


UnboundLocalError: cannot access local variable 'NewX' where it is not associated with a value

In [585]:
espacio.PlayerList

[Player(Name='🤸', Position=[11, 9], BestRoute=[], IsRetard=False, IsDumb=False),
 Player(Name='🤸', Position=[7, 8], BestRoute=[], IsRetard=False, IsDumb=False)]